In [ ]:
import random
import math

# ==========================================
# 0. EL MOTOR MATEMÁTICO (REEMPLAZANDO NUMPY)
# ==========================================

def sigmoid(x):
    return 1 / (1 + math.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

# Función para aplicar Sigmoide a una matriz completa (lista de listas)
def apply_function(matrix, func):
    return [[func(x) for x in row] for row in matrix]

